## Statistics of Cash Flow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the CSV file
df = pd.read_csv("cashflow_1_year_0.csv")

# Clean up numeric columns: remove dots, convert to float
def parse_number(series):
    return series.astype(str).str.replace('.', '', regex=False).astype(float)

# Define the columns that contain income/outcome values
value_columns = [col for col in df.columns if "INCOME" in col or "OUTCOME" in col]

# Convert values from strings to numeric floats
for col in value_columns:
    df[col] = parse_number(df[col])

# Convert PERIOD to datetime (assumes 'Sep-24', etc.)
df['PERIOD'] = pd.to_datetime(df['PERIOD'], format="%b-%y")

# Select only value columns for statistical summary
value_data = df[value_columns]

# Compute standard descriptive statistics
summary_stats = value_data.describe().T

# Rename columns for clarity
summary_stats = summary_stats.rename(columns={
    "mean": "Mean",
    "std": "Standard Deviation",
    "min": "Minimum",
    "25%": "Q1 (25th Percentile)",
    "50%": "Median",
    "75%": "Q3 (75th Percentile)",
    "max": "Maximum"
})

# Add additional insights
summary_stats["Range"] = summary_stats["Maximum"] - summary_stats["Minimum"]
summary_stats["IQR"] = summary_stats["Q3 (75th Percentile)"] - summary_stats["Q1 (25th Percentile)"]
summary_stats["CV (%)"] = (summary_stats["Standard Deviation"] / summary_stats["Mean"]) * 100

# Format summary stats with thousands separator as strings
formatted_summary_stats = summary_stats.copy()
for col in formatted_summary_stats.columns:
    formatted_summary_stats[col] = formatted_summary_stats[col].apply(lambda x: f"{x:,.2f}")

# Optional: Style the full DataFrame (if using Jupyter)
formatted_df = df.style.format({
    col: "{:,.2f}" for col in df.select_dtypes("number").columns if col != "NO"
}).format({
    "NO": "{:.0f}"
})

# ✅ EXPORT to Excel
with pd.ExcelWriter("formatted_cashflow.xlsx", engine="xlsxwriter") as writer:
    # Sheet 1: Cleaned cashflow data
    df.to_excel(writer, sheet_name="Cashflow", index=False)
    workbook  = writer.book
    worksheet = writer.sheets["Cashflow"]

    # Format definitions
    money_fmt = workbook.add_format({'num_format': '#,##0.00'})
    no_fmt = workbook.add_format({'num_format': '0'})

    # Apply formatting to Sheet 1
    worksheet.set_column('A:A', 5, no_fmt)       # NO
    worksheet.set_column('B:B', 12)              # PERIOD
    worksheet.set_column('C:N', 22, money_fmt)   # All income/outcome columns

    # Sheet 2: Formatted summary stats
    formatted_summary_stats.to_excel(writer, sheet_name="Summary Stats")
    stats_sheet = writer.sheets["Summary Stats"]
    stats_sheet.set_column('A:K', 22)            # Adjust width of summary sheet


# Ensure the 'plots' directory exists
os.makedirs("plots", exist_ok=True)

# === 1. LINE PLOT: Time Series for All Programmes ===
plt.figure(figsize=(14, 7))
for col in value_columns:
    plt.plot(df['PERIOD'], df[col], label=col)
plt.title("Income and Outcome Trends Over Time")
plt.xlabel("Period")
plt.ylabel("Amount")
plt.xticks(rotation=45)
plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1))
plt.tight_layout()
lineplot_path = "plots/line_trends.png"
plt.savefig(lineplot_path)
plt.close()

# === 2. BOXPLOT: Distribution of Cashflow Values ===
plt.figure(figsize=(12, 6))
df[value_columns].plot(kind="box", vert=False)
plt.title("Cashflow Distribution per Programme (Income & Outcome)")
plt.xlabel("Amount")
plt.tight_layout()
boxplot_path = "plots/boxplot.png"
plt.savefig(boxplot_path)
plt.close()

<Figure size 1200x600 with 0 Axes>